# Busca de fretes complemento usando linguagem natural

O intuito do estudo é avaliar a viabilidade de realizar uma busca por fretes complementos utilizando uma interface de texto onde o caminhoneiro informa o espaço que possui no caminhão (através de dimensões, volume, peso, etc) e é feito uma busca nos fretes para achar um frete complemento adequado.

Muitos dos fretes possui informações de dimensão, volume e peso no campo de observação, e isso é um dos obstáculos para filtrar fretes. Por isso, aqui mostramos como essas informações também podem ser extraídas dos fretes assim que eles são cadastrados e se tornarem campos estruturados no banco.

Uma vez que conseguimos extrair as informações relevantes do campo de observação e conseguimos entender o espaço disponível no caminhão, podemos recomendar fretes complementos com muito mais facilidade para o caminhoneiro.

In [1]:
import os
import json
from pprint import pprint
from dotenv import load_dotenv
import openai
import pandas as pd

load_dotenv('openai_token.env')

openai.api_key = os.getenv('OPENAI_KEY')
MODEL = "gpt-3.5-turbo"


def convert_weight_units(value, unit):
    supported_units = ['kg', 'T']
    final_unit = 'kg'
    if unit == 'kg' or unit is None:
        return (value, final_unit)
    elif unit == 'T':
        return (value*1000, final_unit)
    else:
        raise ValueError(f'Unit {unit} is not supported. Supported units are {supported_units}')


def convert_volume_units(value, unit):
    supported_units = ['m3', 'cm3']
    final_unit = 'm3'
    if unit == 'm3' or unit is None:
        return (value, final_unit)
    elif unit == 'cm3':
        return (value*1e-6, final_unit)
    else:
        raise ValueError(f'Unit {unit} is not supported. Supported units are {supported_units}')

def convert_length_units(value, unit):
    supported_units = ['m', 'cm']
    final_unit = 'm'
    if unit == 'm' or unit is None:
        return (value, final_unit)
    elif unit == 'cm':
        return (value*1e6, final_unit)
    else:
        raise ValueError(f'Unit {unit} is not supported. Supported units are {supported_units}')


def convert_freight_units(input_record):
    length_fields = ['altura_da_carga', 'comprimento_da_carga', 'largura_da_carga']
    weight_fields = ['peso']
    volume_fields = ['volume_da_carga']
    for field in input_record.keys():
        unit_field = f"unidade_{field}"
        if field in length_fields:
            converted_value, new_unit = convert_length_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
        elif field in weight_fields:
            converted_value, new_unit = convert_weight_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
        elif field in volume_fields:
            converted_value, new_unit = convert_volume_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
    return input_record


def convert_request_units(input_record):
    length_fields = ['altura_maxima_da_carga', 'comprimento_maximo_da_carga', 'largura_maxima_da_carga']
    weight_fields = ['peso_maximo']
    volume_fields = ['volume_maximo_da_carga']
    for field in input_record.keys():
        unit_field = f"unidade_{field}"
        if field in length_fields:
            converted_value, new_unit = convert_length_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
        elif field in weight_fields:
            converted_value, new_unit = convert_weight_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
        elif field in volume_fields:
            converted_value, new_unit = convert_volume_units(input_record[field], input_record[unit_field])
            input_record[field] = converted_value
            input_record[unit_field] = new_unit
    return input_record


def text_to_number(text):
    has_point_or_comma = (('.' in text) or (',' in text))
    if not has_point_or_comma:
        return float(text)
    text = text.replace(',', '.')
    text = text.split('.')
    if len(text[-1]) == 3:
        text = ''.join(text)
    else:
        text = f"{''.join(text[:-1])}.{text[-1]}"

    return float(text)


def convert_text_fields_to_number(input_record, fields):
    for field in fields:
        if isinstance(input_record[field], str):
            input_record[field] = text_to_number(input_record[field])
    return input_record


def calculate_volume(input_record):
    if input_record['volume_da_carga'] is not None:
        return input_record
    
    dimensions_fields = ['comprimento_da_carga', 'largura_da_carga', 'altura_da_carga']
    has_all_dimension_fields = all([True if input_record[field] is not None else False for field in dimensions_fields ])
    if has_all_dimension_fields:
        input_record['volume_da_carga'] = \
            input_record['comprimento_da_carga'] * input_record['largura_da_carga'] * input_record['altura_da_carga']
    return input_record

## Parte 1: Extraindo informações relevantes dos fretes

Nessa etapa, extraímos do texto de descrição do frete informações relevantes de volume, peso e dimensões da carga. Essas informações irão enriquecer o cadastro do frete com outras informações que já estão em campos estruturados.

Todos os fretes de exemplo abaixo foram colocados com mesma cidade de origem, mesma cidade de destino e mesma data de coletam para tornar mais fácil a avaliação dos filtros segundo as informações do campo de observação.

Um outro ponto é que a descrição dos fretes de exemplo utilizados são descrições reais de fretes da plataforma.

In [2]:
def extract_information_from_freight(freight, show_pricing=True):
    prompt = """
    Sua tarefa é extrair da descrição de um frete\
    algumas informações relevantes sobre a carge e estruturar \
    a resposta como um json com os seguintes campos: \
    "peso": "Valor textual do peso da carga sem unidade",
    "unidade_peso": "Valores permitidos: ['kg', 'T']",
    "comprimento_da_carga": "Valor textual de comprimento da carga  sem unidade, geralmente medido em metros",
    "unidade_comprimento_da_carga": "Valores permitidos: ['m', 'cm']",
    "largura_da_carga": "Valor textual de largura da carga  sem unidade, geralmente medido em metros",
    "unidade_largura_da_carga": "Valores permitidos: ['m', 'cm']",
    "altura_da_carga": "Valor textual de altura da carga  sem unidade, geralmente medido em metros",
    "unidade_altura_da_carga": "Valores permitidos: ['m', 'cm']",
    "volume_da_carga": "Valor textual do volume da carga  sem unidade, normalmente medido em metros cúbicos (m3)",
    "unidade_volume_da_carga": "Valores permitidos: ['m3', 'cm3']"
    Caso qualquer a informação de qualquer dos campos não esteja presente \
    na descrição, é importante que você retorne null. Não retorne nada além do \
    json na resposta. É importante que utilize somente os valores permitidos de \
    cada campo.
    Abaixo está a descrição do frete:
    ´´´
    {description}
    ´´´
    """
    final_prompt = prompt.format(description=freight['description'])
    response = openai.ChatCompletion.create(
        model=MODEL,
        temperature=0,
        messages=[{'role': 'user', 'content': final_prompt}]
    )
    extracted_information = json.loads(response["choices"][-1]["message"]["content"])
    return extracted_information

In [3]:
freights_examples = [
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("PESO 17.000 KGS CARREGA DIA 31.07.23 FRETE TOTAL $ 4.000,00 AD. 50,% X 50% "
                     "LIVRE CARGA E DESCARGA CAMINHÃO E EQUIPAMENTO BONS. ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("????????? COMPLEMENTO LIBERADO ?BLUMENAU SC X NOVA MUTUM MT CAMINHÃO ABERTO OU "
                     "FECHADO BRINQUEDOS PARA PARQUINHOS PESO 300 KG TOTAL PODE CARREGAR POR CIMA DE "
                     "OUTRA CARGA LIVRE DE CARGA E DESCARGA R$ 600 + 50 DE PEDAGIO ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("6 TON COLETA AMANHÃ PELA MANHÃ SOMENTE SIDER 15,40 ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("COMPLEMENTO LIBERADO ?SÃO BENTO DO SUL SC X QUERÊNCIA MT CAMINHÃO DE 11 MT ABERTO "
                     "OU FECHADO CHAPAS DE TELHA TRANSLÚCIDA C- 11.00 L- 1.20 A- 0,40 PESO 724 KG TOTAL "
                     "PODE CARREGAR POR CIMA DE OUTRA CARGA LIVRE DE CARGA E DESCARGA R$ 1.200 + 100 DE PEDAGIO ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("1 VOL LIVR DE CARGA E DESCARGA MEDIDAS 1,50 ALTURA X 1 LARGURA X 2 MT COMPRIMENTO ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("3.300 KG...MATERIAL VAI EM CIMA DE CARGA....CARREGAMENTO LIBERADO ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("1,5 MTS. CÚBICOS DE MUDANÇA DESMONTADA E EMBALADA NO DEPOSITO --- PESO: +/- 600 KGS -- DESCARGA POR CONTA DO MOTORISTA -- PREFERÊNCIA PARA MENSAGEM VIA WHATSAPP COM O VALDEZ ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("4 TON LIVRE DE DESCARGA ")},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": ("blablabla")},
]

freights_extracted_information = []
numeric_fields = ['peso', 'comprimento_da_carga', 'largura_da_carga', 
                  'altura_da_carga', 'volume_da_carga']
for i, freight in enumerate(freights_examples):
    extracted_information = extract_information_from_freight(freight)
    extracted_information = convert_text_fields_to_number(extracted_information,
                                                          fields=numeric_fields)
    extracted_information = convert_freight_units(extracted_information)
    extracted_information = calculate_volume(extracted_information)

    print(f"\n\n#### Exemplo {i+1} ####")
    print("\nFrete:")
    pprint(freight)
    print("\nInformações extraídas do frete:")
    pprint(extracted_information)

    extracted_information.update(freight)
    freights_extracted_information.append(extracted_information)
    



#### Exemplo 1 ####

Frete:
{'cidade_destino_id': 3,
 'cidade_origem_id': 2,
 'data_carregamento': '15/08/2023',
 'description': 'PESO 17.000 KGS CARREGA DIA 31.07.23 FRETE TOTAL $ 4.000,00 '
                'AD. 50,% X 50% LIVRE CARGA E DESCARGA CAMINHÃO E EQUIPAMENTO '
                'BONS. '}

Informações extraídas do frete:
{'altura_da_carga': None,
 'comprimento_da_carga': None,
 'largura_da_carga': None,
 'peso': 17000.0,
 'unidade_altura_da_carga': 'm',
 'unidade_comprimento_da_carga': 'm',
 'unidade_largura_da_carga': 'm',
 'unidade_peso': 'kg',
 'unidade_volume_da_carga': 'm3',
 'volume_da_carga': None}


#### Exemplo 2 ####

Frete:
{'cidade_destino_id': 3,
 'cidade_origem_id': 2,
 'data_carregamento': '15/08/2023',
 'description': '????????? COMPLEMENTO LIBERADO ?BLUMENAU SC X NOVA MUTUM MT '
                'CAMINHÃO ABERTO OU FECHADO BRINQUEDOS PARA PARQUINHOS PESO '
                '300 KG TOTAL PODE CARREGAR POR CIMA DE OUTRA CARGA LIVRE DE '
                'CARGA E D

In [4]:
database = pd.DataFrame(freights_extracted_information)
database

,peso,unidade_peso,comprimento_da_carga,unidade_comprimento_da_carga,largura_da_carga,unidade_largura_da_carga,altura_da_carga,unidade_altura_da_carga,volume_da_carga,unidade_volume_da_carga,cidade_origem_id,cidade_destino_id,data_carregamento,description
0,17000.0,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,PESO 17.000 KGS CARREGA DIA 31.07.23 FRETE TOT...
1,300.0,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,????????? COMPLEMENTO LIBERADO ?BLUMENAU SC X ...
2,6000.0,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,6 TON COLETA AMANHÃ PELA MANHÃ SOMENTE SIDER 1...
3,724.0,kg,11.0,m,1.2,m,0.4,m,5.28,m3,2,3,15/08/2023,COMPLEMENTO LIBERADO ?SÃO BENTO DO SUL SC X QU...
4,NaN,kg,2.0,m,1.0,m,1.5,m,3.00,m3,2,3,15/08/2023,"1 VOL LIVR DE CARGA E DESCARGA MEDIDAS 1,50 AL..."
5,3300.0,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,3.300 KG...MATERIAL VAI EM CIMA DE CARGA....CA...
6,600.0,kg,NaN,m,NaN,m,NaN,m,1.50,m3,2,3,15/08/2023,"1,5 MTS. CÚBICOS DE MUDANÇA DESMONTADA E EMBAL..."
7,4000.0,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,4 TON LIVRE DE DESCARGA
8,NaN,kg,NaN,m,NaN,m,NaN,m,NaN,m3,2,3,15/08/2023,blablabla


## Parte 2: Extraindo requisitos do caminhoneiro

Nessa etapa, precisamos extrair os requisitos do frete do texto de descrição do frete complemento desejado escrito pelo caminhoneiro.

Simulamos alguns textos com formas de como o caminhoneiro poderia especificar o espaço disponível no seu caminhão em um campo de texto aberto.

In [5]:
def extract_information_from_request(request):
    prompt = """
    Sua tarefa é extrair de um texto de solicitação \
    de um caminhoneiro algumas informações e estruturar \
    a resposta como json com a seguinte estrutura:
    "peso_maximo" : "Valor textual do peso máximo que a carga pode ter sem unidade",
    "unidade_peso_maximo": "Valores permitidos: ['kg', 'T']",
    "comprimento_maximo_da_carga": "Valor textual do comprimento máximo da carga sem unidade, geralmente medido em metros",
    "unidade_comprimento_maximo_da_carga": "Valores permitidos: ['m', 'cm']",
    "largura_maxima_da_carga": "Valor textual da largura máxima da carga sem unidade, geralmente medido em metros",
    "unidade_largura_maxima_da_carga": "Valores permitidos: ['m', 'cm']",
    "altura_maxima_da_carga": "Valor textual da altura da carga sem unidade, geralmente medido em metros",
    "unidade_altura_maxima_da_carga": "Valores permitidos: ['m', 'cm']",
    "volume_maximo_da_carga": "Valor textual do volume máximo da carga sem unidade",
    "unidade_volume_maximo_da_carga": "Valores permitidos: ['m3', 'cm3']"

    Caso qualquer a informação de qualquer dos campos não esteja presente \
    na descrição, é importante que você retorne null. Não retorne nada além do \
    json na resposta. É importante que utilize somente os valores permitidos de \
    cada campo. Abaixo está a solicitação do caminhoneiro:
    ´´´
    {description}
    ´´´
    """
    final_prompt = prompt.format(description=request['description'])
    response = openai.ChatCompletion.create(
        model=MODEL,
        temperature=0,
        messages=[{'role': 'user', 'content': final_prompt}]
    )
    extracted_requirements = json.loads(response["choices"][-1]["message"]["content"])
    return extracted_requirements

In [6]:
requests_examples = [
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": "Quero um frete com até 2T"},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": "Frete complemento com no maximo 20m3"},
    {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": "Olá, queria um complemento pequeno de até uns 700 quilos e volume de ate 20 mts cubico"},
     {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": "Meu caminhão já tá meio cheio pode só carga de no maximo 10m3"},
     {"cidade_origem_id": 2,
     "cidade_destino_id": 3,
     "data_carregamento": "15/08/2023",
     "description": "Carga pequena até 1.5 metrus de largura e 3 de comprimento"},
]

resquest_extracted_requirements = []
numeric_fields = ["peso_maximo", "comprimento_maximo_da_carga",
                  "largura_maxima_da_carga", "altura_maxima_da_carga",
                  "volume_maximo_da_carga"]
for i, request in enumerate(requests_examples):
    extracted_information = extract_information_from_request(request)
    extracted_information = convert_text_fields_to_number(extracted_information,
                                                          fields=numeric_fields)
    extracted_information = convert_request_units(extracted_information)
    resquest_extracted_requirements.append(extracted_information)

    print(f"\n\n#### Exemplo {i+1} ####")
    print("\Requisitos do caminhoneiro com relação ao frete:")
    pprint(request)
    print("\nRequisitos extraídos da requisição do caminhoneiro:")
    pprint(extracted_information)



#### Exemplo 1 ####
\Requisitos do caminhoneiro com relação ao frete:
{'cidade_destino_id': 3,
 'cidade_origem_id': 2,
 'data_carregamento': '15/08/2023',
 'description': 'Quero um frete com até 2T'}

Requisitos extraídos da requisição do caminhoneiro:
{'altura_maxima_da_carga': None,
 'comprimento_maximo_da_carga': None,
 'largura_maxima_da_carga': None,
 'peso_maximo': 2000.0,
 'unidade_altura_maxima_da_carga': 'm',
 'unidade_comprimento_maximo_da_carga': 'm',
 'unidade_largura_maxima_da_carga': 'm',
 'unidade_peso_maximo': 'kg',
 'unidade_volume_maximo_da_carga': 'm3',
 'volume_maximo_da_carga': None}


#### Exemplo 2 ####
\Requisitos do caminhoneiro com relação ao frete:
{'cidade_destino_id': 3,
 'cidade_origem_id': 2,
 'data_carregamento': '15/08/2023',
 'description': 'Frete complemento com no maximo 20m3'}

Requisitos extraídos da requisição do caminhoneiro:
{'altura_maxima_da_carga': None,
 'comprimento_maximo_da_carga': None,
 'largura_maxima_da_carga': None,
 'peso_maximo':

## Etapa 3: Encontrando fretes relevantes

Nessa última etapa, buscamos fretes que sejam relevantes de acordo com a solicitação de cada caminhoneiro.

In [7]:
def recommend_freights(request, database):
    request_value_fields = ['peso_maximo', 
                        'comprimento_maximo_da_carga', 
                        'largura_maxima_da_carga', 
                        'altura_maxima_da_carga', 
                        'volume_maximo_da_carga']
    request_field_to_database_field ={
        'peso_maximo':'peso',
        'comprimento_maximo_da_carga': 'comprimento_da_carga',
        'largura_maxima_da_carga': 'largura_da_carga',
        'altura_maxima_da_carga': 'altura_da_carga',
        'volume_maximo_da_carga': 'volume_da_carga'
    }
    filters = []
    for field in request_value_fields:
        if request[field] is not None:
            filters.append(f"{request_field_to_database_field[field]} <= {request[field]}")
    final_filters = ' and '.join(filters)

    recommended_freights = database.query(final_filters)
    freights_returned_fields = ['cidade_origem_id', 
                                'cidade_destino_id', 
                                'data_carregamento', 
                                'description']
    return recommended_freights[freights_returned_fields].to_dict(orient='records')

In [8]:
for i, (request, request_requirements) in enumerate(zip(requests_examples, resquest_extracted_requirements)):
    recommendations = recommend_freights(request_requirements, database)
    print(f"\n\n#### Exemplo {i+1} ####")
    print("\nRequisitos do frete complemento do caminhoneiro:")
    pprint(request)
    print("\nFretes complemento encontrados:")
    if recommendations:
        pprint(recommendations)
    else:
        print("Nenhum frete encontrado com essas características. :/")



#### Exemplo 1 ####

Requisitos do frete complemento do caminhoneiro:
{'cidade_destino_id': 3,
 'cidade_origem_id': 2,
 'data_carregamento': '15/08/2023',
 'description': 'Quero um frete com até 2T'}

Fretes complemento encontrados:
[{'cidade_destino_id': 3,
  'cidade_origem_id': 2,
  'data_carregamento': '15/08/2023',
  'description': '????????? COMPLEMENTO LIBERADO ?BLUMENAU SC X NOVA MUTUM MT '
                 'CAMINHÃO ABERTO OU FECHADO BRINQUEDOS PARA PARQUINHOS PESO '
                 '300 KG TOTAL PODE CARREGAR POR CIMA DE OUTRA CARGA LIVRE DE '
                 'CARGA E DESCARGA R$ 600 + 50 DE PEDAGIO '},
 {'cidade_destino_id': 3,
  'cidade_origem_id': 2,
  'data_carregamento': '15/08/2023',
  'description': 'COMPLEMENTO LIBERADO ?SÃO BENTO DO SUL SC X QUERÊNCIA MT '
                 'CAMINHÃO DE 11 MT ABERTO OU FECHADO CHAPAS DE TELHA '
                 'TRANSLÚCIDA C- 11.00 L- 1.20 A- 0,40 PESO 724 KG TOTAL PODE '
                 'CARREGAR POR CIMA DE OUTRA CARGA LIVRE DE